In [1]:
import torch
import torch.nn.functional as F
from torch.utils import data
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim
import sys
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm
from datetime import datetime
import copy
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [2]:
import platform; print(platform.platform())

macOS-14.6-arm64-arm-64bit


In [3]:
df = pd.read_csv('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/labelled_data_500_30bps.csv')

In [4]:
x = pd.to_datetime(df['date']).value_counts()
x.sort_index().index

DatetimeIndex(['2021-06-01', '2021-06-02', '2021-06-03', '2021-06-04',
               '2021-06-07', '2021-06-08', '2021-06-09', '2021-06-10',
               '2021-06-11', '2021-06-15', '2021-06-16', '2021-06-17',
               '2021-06-18', '2021-06-21', '2021-06-22'],
              dtype='datetime64[ns]', name='date', freq=None)

In [5]:
train_set = df[pd.to_datetime(df['date']) <= pd.to_datetime('2021-06-07')]
test_data = df[pd.to_datetime(df['date']) == pd.to_datetime('2021-06-08')]
train_set.head()

,date,SP1,SV1,BP1,BV1,SP2,SV2,BP2,BV2,SP3,...,BV3,SP4,SV4,BP4,BV4,SP5,SV5,BP5,BV5,label
0,2021-06-01,10200.0,3294,10150.0,61,10250.0,2185,10100.0,1240,10300.0,...,1403,10350.0,2138,10000.0,2195,10400.0,2596,9990.0,685,1
1,2021-06-01,10200.0,3294,10150.0,59,10250.0,2185,10100.0,1240,10300.0,...,1403,10350.0,2138,10000.0,2195,10400.0,2596,9990.0,685,1
2,2021-06-01,10200.0,3324,10150.0,59,10250.0,2185,10100.0,1240,10300.0,...,1403,10350.0,2138,10000.0,2195,10400.0,2596,9990.0,685,1
3,2021-06-01,10200.0,3324,10150.0,43,10250.0,2185,10100.0,1240,10300.0,...,1403,10350.0,2138,10000.0,2195,10400.0,2596,9990.0,685,1
4,2021-06-01,10200.0,3324,10150.0,43,10250.0,2185,10100.0,1240,10300.0,...,1403,10350.0,2138,10000.0,2195,10400.0,2596,9990.0,687,1


In [6]:
train_data =  train_set.iloc[:int(np.floor(train_set.shape[0] * 0.8)),:]
eval_data = train_set.iloc[int(np.floor(train_set.shape[0] * 0.8)):,:]

In [7]:
def normalize_data(df,cols,norm):
    #Normalizing using z-score
    if norm=='Z':
        scaler = StandardScaler()
        scaler.fit(df[cols])
        data = scaler.transform(df[cols])
    
    
    #Normalizing using DecPrec
    if norm=='DecPrec':
        k_len = np.ceil(np.log10(df[cols].abs().max()))
        # print(k_len)
        data = df[cols]/(10**k_len)

    return data,scaler

In [8]:
train_input , scaler = normalize_data(train_data.iloc[:,1:-1],train_data.iloc[:,1:-1].columns,'Z')
train_label = train_data.iloc[:,-1].to_numpy()
eval_input = scaler.transform(eval_data.iloc[:,1:-1])
eval_label = eval_data.iloc[:,-1].to_numpy()
test_input = scaler.transform(test_data.iloc[:,1:-1])
test_label = test_data.iloc[:,-1].to_numpy()

In [9]:
del [train_data,test_data,eval_data,df,train_set]

In [10]:
s=0
cc=0
key=0
obj=0
for key,obj in locals().items():
    cc= sys.getsizeof(obj)/1000000
    s+=cc
    if cc>20:
        print(key,cc)
print(s,'mb')

0.015917 mb


In [11]:
np.isnan(eval_input).any()

False

In [12]:
class Dataset_LOB(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, x,y, num_classes, T):
        """Initialization""" 
        # self.k = k
        self.num_classes = num_classes
        self.T = T
            
        # x = prepare_x(data)
        # y = get_label(data)
        x, y = data_classification(x, y, self.T)
        # y = y[:,self.k] - 1
        self.length = len(x)

        x = torch.from_numpy(x)
        self.x = torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index]

In [13]:
def data_classification(X, Y, T):
    [N, D] = X.shape
    # print(X.shape,T,N,Y.shape)
    df = np.array(X)
    # print(df.shape)
    dY = np.array(Y)

    dataY = dY[T - 1:N]

    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]

    return dataX, dataY


In [14]:
batch_size = 64
dataset_train = Dataset_LOB(train_input,train_label, num_classes=3, T=100)
dataset_eval = Dataset_LOB(eval_input,eval_label, num_classes=3, T=100)
dataset_test = Dataset_LOB(test_input,test_label, num_classes=3, T=100)

# test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

In [15]:
train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_eval, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)
print(dataset_train.x.shape, dataset_test.y.shape)

torch.Size([1732671, 1, 100, 20]) torch.Size([282495])


In [16]:
tmp_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=1, shuffle=True)

for x, y in tmp_loader:
    print(x)
    print(y)
    print(x.shape, y.shape)
    break

tensor([[[[ 0.8504, -0.9396,  0.8519,  ...,  0.0262,  0.8641,  0.2516],
          [ 0.8504, -0.9396,  0.8519,  ...,  0.0262,  0.8641,  0.2516],
          [ 0.8504, -0.9396,  0.8519,  ...,  0.0262,  0.8641,  0.2516],
          ...,
          [ 0.8504, -0.7872,  0.8519,  ...,  0.0262,  0.8641,  0.2516],
          [ 0.8504, -0.7869,  0.8519,  ...,  0.0262,  0.8641,  0.2516],
          [ 0.8504, -0.8168,  0.8519,  ...,  0.0262,  0.8641,  0.2516]]]],
       dtype=torch.float64)
tensor([1])
torch.Size([1, 1, 100, 20]) torch.Size([1])


In [17]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

device = mps_device

tensor([1.], device='mps:0')


In [18]:
class deeplob(nn.Module):
    def __init__(self, y_len):
        super().__init__()
        self.y_len = y_len
        
        # convolution blocks
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.LeakyReLU(negative_slope=0.01),
#             nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,5)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        
        # inception moduels
        self.inp1 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inp2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        self.inp3 = nn.Sequential(
            nn.MaxPool2d((3, 1), stride=(1, 1), padding=(1, 0)),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
        )
        
        # lstm layers
        self.lstm = nn.LSTM(input_size=192, hidden_size=64, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(64, self.y_len)

    def forward(self, x):
        # h0: (number of hidden layers, batch size, hidden size)
        h0 = torch.zeros(1, x.size(0), 64).to(device)
        c0 = torch.zeros(1, x.size(0), 64).to(device)
    
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x_inp1 = self.inp1(x)
        x_inp2 = self.inp2(x)
        x_inp3 = self.inp3(x)  
        
        x = torch.cat((x_inp1, x_inp2, x_inp3), dim=1)
        
#         x = torch.transpose(x, 1, 2)
        x = x.permute(0, 2, 1, 3).contiguous()
        x = torch.reshape(x, (-1, x.shape[1], x.shape[2]))
        
        x, _ = self.lstm(x, (h0, c0))
        x = x[:, -1, :]
        x = self.fc1(x)
        forecast_y = torch.softmax(x, dim=1)
        
        return forecast_y

In [19]:
model = deeplob(y_len = dataset_train.num_classes)
model.to(mps_device)
# device = mps_device

deeplob(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(1, 2), stride=(1, 2))
    (1): LeakyReLU(negative_slope=0.01)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(4, 1), stride=(1, 1))
    (4): LeakyReLU(negative_slope=0.01)
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(32, 32, kernel_size=(4, 1), stride=(1, 1))
    (7): LeakyReLU(negative_slope=0.01)
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(1, 2), stride=(1, 2))
    (1): Tanh()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(4, 1), stride=(1, 1))
    (4): Tanh()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(32, 32, kernel_size=(4, 1), stride

In [20]:
device

device(type='mps')

In [23]:
class_count = Counter(train_label)
class_weights = torch.Tensor([len(train_label)/c for c in pd.Series(class_count).sort_index().values])
class_weights = class_weights.to(device)  
criterion = nn.CrossEntropyLoss(class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [24]:
class_weights

tensor([35.9862,  1.0569, 38.3458], device='mps:0')

In [25]:
class EarlyStopping:
    def __init__(self, patience=1, path='./model.pt'):
        self.patience = patience
        self.path= path
        self.counter = 0
        self.best_score = np.inf
        self.early_stop = False
        self.best_test_epoch = 0
        
    def __call__(self, val_loss, model,it):
        if self.best_score is None:
            self.best_score = val_loss
            self.save_checkpoint(model)
            self.best_test_epoch = it
    
        elif val_loss > self.best_score:
            self.counter +=1
            if self.counter >= self.patience:
                self.early_stop = True 
        else:
            self.best_score = val_loss
            self.save_checkpoint(model)
            self.counter = 0
            self.best_test_epoch = it

    def save_checkpoint(self, model):
        torch.save(model.state_dict(), self.path)

In [26]:
checkpoint_dir = '/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/fast-alphas/best_val_roLOB_30bps.pt'

In [27]:
early_stopping = EarlyStopping(
        patience=7, 
        path=checkpoint_dir)

In [28]:
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs):

    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)


    for it in tqdm(range(epochs)):
        
        model.train()
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            # move data to GPU
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

            # zero the parameter gradients
            optimizer.zero_grad()
            # Forward pass
            # print("about to get model output")
            outputs = model(inputs)
            # print("done getting model output")
            # print("outputs.shape:", outputs.shape, "targets.shape:", targets.shape)
            loss = criterion(outputs, targets)
            # Backward and optimize
            # print("about to optimize")
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
        # Get train loss and test loss
        train_loss = np.mean(train_loss) # a little misleading
    
        model.eval()
        test_loss = []
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)      
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss.append(loss.item())
        test_loss = np.mean(test_loss)

        # Save losses
        train_losses[it] = train_loss
        test_losses[it] = test_loss
        
        early_stopping(test_loss,model,it)
        if early_stopping.early_stop:
            print('Patience Exhausted')
            break

        dt = datetime.now() - t0
        print(dt)
        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
          Validation Loss: {test_loss:.4f}, Duration: {dt}, Best Val Epoch: {early_stopping.best_test_epoch}')

    return train_losses, test_losses

In [29]:
train_losses, val_losses = batch_gd(model, criterion, optimizer, 
                                    train_loader, val_loader, epochs=25)

  4%|█▌                                     | 1/25 [26:14<10:29:42, 1574.27s/it]

0:26:14.269653
Epoch 1/25, Train Loss: 0.8045,           Validation Loss: 0.8274, Duration: 0:26:14.269653, Best Val Epoch: 0


  8%|███                                    | 2/25 [53:08<10:12:31, 1597.88s/it]

0:26:54.391658
Epoch 2/25, Train Loss: 0.7295,           Validation Loss: 0.8515, Duration: 0:26:54.391658, Best Val Epoch: 0


 12%|████▌                                 | 3/25 [1:20:07<9:49:25, 1607.52s/it]

...

0:27:20.015649
Epoch 21/25, Train Loss: 0.6220,           Validation Loss: 0.7647, Duration: 0:27:20.015649, Best Val Epoch: 19


 88%|████████████████████████████████▌    | 22/25 [9:58:33<1:22:20, 1646.79s/it]

0:27:39.860612
Epoch 22/25, Train Loss: 0.6208,           Validation Loss: 0.7817, Duration: 0:27:39.860612, Best Val Epoch: 19


 88%|███████████████████████████████▋    | 22/25 [10:16:11<1:24:01, 1680.52s/it]


KeyboardInterrupt: 

In [ ]:
early_stopping.best_test_epoch

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(train_losses, label='train loss')
plt.plot(val_losses, label='validation loss')
plt.legend()

In [ ]:

n_correct = 0.
n_total = 0.
all_targets = []
all_predictions = []

for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    # update counts
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

test_acc = n_correct / n_total
print(f"Test acc: {test_acc:.4f}")

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)   

In [ ]:
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

In [21]:
from sklearn.metrics import confusion_matrix

In [22]:
model.load_state_dict(torch.load('/Users/jandh/Desktop/Old Desktop/od/1 quater/Project Lab/fast-alphas/best_val_roLOB_30bps.pt', map_location=mps_device))

/var/folders/vf/gw1t6f4j3mv84kp8kpw4j_bw0000gn/T/ipykernel_94672/40889344.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/Users/jandh/

<All keys matched successfully>

# Training with adjusted class frequncy weighted loss function

In [23]:
n_correct = 0.
n_total = 0.
all_targets = []
all_predictions = []

for inputs, targets in test_loader:
    # Move to GPU
    model.eval()
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    # update counts
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

test_acc = n_correct / n_total
print(f"Test acc: {test_acc:.4f}")

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)   

Test acc: 0.6893


In [24]:
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

accuracy_score: 0.689318395015841
              precision    recall  f1-score   support

           0     0.0794    0.3257    0.1276     10760
           1     0.9424    0.7152    0.8132    262937
           2     0.0819    0.3611    0.1335      8798

    accuracy                         0.6893    282495
   macro avg     0.3679    0.4673    0.3581    282495
weighted avg     0.8827    0.6893    0.7659    282495



In [25]:
confusion_matrix(all_targets,all_predictions)

array([[  3505,   6239,   1016],
       [ 40301, 188047,  34589],
       [   358,   5263,   3177]])

# Train data Stats

In [33]:
n_correct = 0.
n_total = 0.
all_targets = []
all_predictions = []

for inputs, targets in train_loader:
    # Move to GPU
    model.eval()
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    # update counts
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

test_acc = n_correct / n_total
print(f"Test acc: {test_acc:.4f}")

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)   

Test acc: 0.8945


In [34]:
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

accuracy_score: 0.8944854504981038
              precision    recall  f1-score   support

           0     0.3534    0.9700    0.5181     48093
           1     0.9982    0.8901    0.9411   1639390
           2     0.3166    0.9725    0.4776     45188

    accuracy                         0.8945   1732671
   macro avg     0.5561    0.9442    0.6456   1732671
weighted avg     0.9625    0.8945    0.9172   1732671



In [35]:
confusion_matrix(all_targets,all_predictions)

array([[  46648,    1444,       1],
       [  85257, 1459254,   94879],
       [     75,    1166,   43947]])

In [36]:
n_correct = 0.
n_total = 0.
all_targets = []
all_predictions = []

for inputs, targets in val_loader:
    # Move to GPU
    model.eval()
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    # update counts
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

test_acc = n_correct / n_total
print(f"Test acc: {test_acc:.4f}")

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)   

Test acc: 0.8009


In [37]:
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

accuracy_score: 0.8009185072986464
              precision    recall  f1-score   support

           0     0.1453    0.2947    0.1946     19166
           1     0.9171    0.8586    0.8869    391548
           2     0.1823    0.2252    0.2015     22380

    accuracy                         0.8009    433094
   macro avg     0.4149    0.4595    0.4277    433094
weighted avg     0.8450    0.8009    0.8208    433094

